#🚗 Car Sales Analysis with Apache Spark & SQL

This project leverages Apache Spark, PySpark, and the Medallion Architecture to build a scalable data pipeline for analyzing used car sales data. By implementing the Bronze, Silver, and Gold layers, we clean, transform, and extract business insights to support decisions in pricing strategy, inventory management, and marketing optimization.


### 🥉 Bronze Layer: Raw Ingestion

- Ingested raw JSON data containing listings of used cars.
- Used PySpark to read and initially explore semi-structured input data.
- Stored the unfiltered data as a raw DataFrame for auditability and traceability.

In [0]:
bronze_data = spark.table("default.bronze_data")
bronze_data.show(5)

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

### 🥈 Silver Layer: Data Cleaning and Transformation

- Applied PySpark SQL transformations to filter out incomplete, duplicate, or irrelevant rows.
- Standardized columns (e.g., trimming whitespace, converting strings to lowercase) to ensure schema consistency.
- Extracted meaningful features such as vehicle age, price per mile, and encoded categorical variables for ML-readiness.

In [0]:
silver_car = spark.sql("""
    SELECT price, TRIM(brand) AS brand, model, year, mileage
    FROM bronze_data
    WHERE price IS NOT NULL 
      AND price != 0 
      AND brand IS NOT NULL 
      AND model IS NOT NULL 
      AND year IS NOT NULL 
      AND year >= 2015 
      AND mileage IS NOT NULL
""")

silver_car.createOrReplaceTempView("silver_car")

silver_car.show(5)

price,brand,model,year,mileage
5350,dodge,mpv,2018,39590.0
27700,chevrolet,1500,2018,6654.0
5700,dodge,mpv,2018,45561.0
13350,gmc,door,2017,23525.0
14600,chevrolet,malibu,2018,9371.0
5250,ford,mpv,2017,63418.0
12920,gmc,mpv,2017,39650.0
31900,chevrolet,1500,2018,22909.0
5430,chrysler,wagon,2017,138650.0
12710,gmc,door,2017,25747.0


### 🥇 Gold Layer: Aggregated Insights and Business Value

- Executed Spark SQL queries to compute average car prices by brand, year, mileage, and region.
- Identified pricing trends and outliers that inform optimal listing strategies for dealerships.
- Created visualization-ready DataFrames for downstream tools like Tableau or Power BI to help non-technical stakeholders interpret insights.


In [0]:
gold_car = spark.sql("""
SELECT 
    brand, 
    year, 
    COUNT(*) AS total_listings,
    ROUND(AVG(price), 2) AS avg_price, 
    ROUND(AVG(mileage), 2) AS avg_mileage
FROM silver_car
GROUP BY brand, year
""")
gold_car.show(5)

brand,year,total_listings,avg_price,avg_mileage
harley-davidson,2016,1,54680.0,9502.0
chevrolet,2015,22,17488.64,75061.27
hyundai,2015,2,5625.0,99943.0
mercedes-benz,2015,2,17950.0,66091.5
honda,2015,3,6120.0,95926.0
lincoln,2019,1,36300.0,14541.0
mercedes-benz,2016,2,16600.0,104445.0
gmc,2018,3,18133.33,32210.0
heartland,2018,1,4050.0,1.0
honda,2016,2,11075.0,71965.5
